In [55]:
import os
import sys
import librosa
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Reshape, Flatten
import keras
import seaborn as sns
import pydub
import pickle
import librosa.display
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
sys.path.append('data_prep/')
import speech_data
import segment_data

In [136]:
# imports
training = '/home/cc/threshold/'
validation = '/home/cc/Data/speaker10Val/'
num_speakers = 10

In [137]:
# calculate the mfcc, delta, delta-delta matrices for training
X = []
Y = []
speakers = speech_data.get_speakers(training)
hop_length = 128 # default = 512
for f in os.listdir(training):
    Y.append(speech_data.one_hot_from_item(speech_data.speaker(f), speakers))
    y, sr = librosa.load(training + f)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20, hop_length=hop_length)
    delta = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)
    X.append(np.c_[mfcc,delta,delta2])
#pickle.dump(X, open('/home/cc/pickle_files/speaker251_train_2secX.p', 'wb'))
#pickle.dump(Y, open('/home/cc/pickle_files/speaker251_train_2secY.p', 'wb'))

number of files: 123
(2, ' speakers: ', ['arun', 'andrew'])


In [112]:
# calculate the mfcc, delta, delta-delta matrices for validation
Xval = []
Yval = []
speakers = speech_data.get_speakers(validation)
hop_length = 128 # default = 512
for f in os.listdir(validation):
    Yval.append(speech_data.one_hot_from_item(speech_data.speaker(f), speakers))
    y, sr = librosa.load(validation + f)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20, hop_length=hop_length)
    delta = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)
    Xval.append(np.c_[mfcc,delta,delta2])
#pickle.dump(Xval, open('/home/cc/pickle_files/speaker251_val_2secX.p', 'wb'))
#pickle.dump(Yval, open('/home/cc/pickle_files/speaker251_val_2secY.p', 'wb'))

number of files: 86
(10, ' speakers: ', ['Speaker1', 'Speaker0', 'Speaker3', 'Speaker2', 'Speaker5', 'Speaker4', 'Speaker7', 'Speaker6', 'Speaker9', 'Speaker8'])


In [138]:
speakers = speech_data.get_speakers(training)
#X = pickle.load(open('/home/cc/pickle_files/speaker40_train_2secX.p', 'rb'))
#Y = pickle.load(open('/home/cc/pickle_files/speaker40_train_2secY.p', 'rb'))
#Xval = pickle.load(open('/home/cc/pickle_files/speaker40_val_2secX.p', 'rb'))
#Yval = pickle.load(open('/home/cc/pickle_files/speaker40_val_2secY.p', 'rb'))
x_train = np.array(X)
y_train = np.array(Y)
x_val = np.array(Xval)
y_val = np.array(Yval)
x_train.shape

number of files: 123
(2, ' speakers: ', ['arun', 'andrew'])


(123, 20, 519)

In [151]:
# prepare the keras model - using Sequential for a linear stack of layers
model = Sequential()

In [152]:
# import / add layers to the model
# start with a dense layer - fully connected
model.add(Dense(units=128, activation='tanh', input_shape=(20, 519,)))
model.add(Dense(units=128, activation='tanh'))
model.add(Dense(units=128, activation='tanh'))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(units=2, activation='softmax'))

In [153]:
# configure model's learning process
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# create the callback functions
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=1, mode='max')
csv_logger = keras.callbacks.CSVLogger('training.log')

In [154]:
# now fit to model
#model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val), callbacks=[csv_logger])
model.fit(x_train, y_train, epochs=10, validation_split=0.1, callbacks=[csv_logger])

Train on 110 samples, validate on 13 samples
Epoch 1/10
110/110 [==============================] - 0s - loss: 0.7680 - acc: 0.5545 - val_loss: 0.4232 - val_acc: 0.8462
Epoch 2/10
110/110 [==============================] - 0s - loss: 0.1975 - acc: 0.9636 - val_loss: 0.3738 - val_acc: 0.8462
Epoch 3/10
110/110 [==============================] - 0s - loss: 0.0893 - acc: 0.9818 - val_loss: 0.2898 - val_acc: 0.8462
Epoch 4/10
110/110 [==============================] - 0s - loss: 0.0438 - acc: 1.0000 - val_loss: 0.2220 - val_acc: 0.7692
Epoch 5/10
110/110 [==============================] - 0s - loss: 0.0226 - acc: 1.0000 - val_loss: 0.1764 - val_acc: 0.9231
Epoch 6/10
110/110 [==============================] - 0s - loss: 0.0102 - acc: 1.0000 - val_loss: 0.1589 - val_acc: 0.9231
Epoch 7/10
110/110 [==============================] - 0s - loss: 0.0089 - acc: 1.0000 - val_loss: 0.1482 - val_acc: 1.0000
Epoch 8/10
110/110 [==============================] - 0s - loss: 0.0086 - acc: 1.0000 - val_lo

In [155]:
model.summary()
print "Inputs: {}".format(model.input_shape)
print "Outputs: {}".format(model.output_shape)
print "Actual input: {}".format(x_train.shape)
print "Actual output: {}".format(y_train.shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_163 (Dense)            (None, 20, 128)           66560     
_________________________________________________________________
dense_164 (Dense)            (None, 20, 128)           16512     
_________________________________________________________________
dense_165 (Dense)            (None, 20, 128)           16512     
_________________________________________________________________
dropout_34 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
flatten_33 (Flatten)         (None, 2560)              0         
_________________________________________________________________
dense_166 (Dense)            (None, 2)                 5122      
Total params: 104,706
Trainable params: 104,706
Non-trainable params: 0
_________________________________________________________________
Inpu

In [157]:
# now to evaluate the model
testing = '/home/cc/Data/ociTest//'
# calculate the mfcc matrices for testing
Xtest = []
Ytest = []
speakers = speech_data.get_speakers(testing)
hop_length = 128
for f in os.listdir(testing):
    Ytest.append(speech_data.one_hot_from_item(speech_data.speaker(f), speakers))
    y, sr = librosa.load(testing + f)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20, hop_length=hop_length)
    delta = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)
    Xtest.append(np.c_[mfcc,delta,delta2])

number of files: 15
(2, ' speakers: ', ['arun', 'andrew'])


In [158]:
x_test = np.array(Xtest)
y_test = np.array(Ytest)
model.evaluate(x_test, y_test)

15/15 [==============================] - 0s


[0.17391367256641388, 0.86666667461395264]

In [135]:
model.metrics_names

['loss', 'acc']